# 3. Анализ результатов

Установка необходимых пакетов

In [1]:
!pip install numpy==1.19.4
!pip install scikit-learn==0.24.0
!pip install scipy==1.5.4
!pip install pandas==1.1.5

  Using cached numpy-1.19.4-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
     |████████████████████████████████| 24.9 MB 2.3 MB/s eta 0:00:01
  Using cached joblib-1.0.0-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached scipy-1.5.4-cp38-cp38-manylinux1_x86_64.whl (25.8 MB)
  Using cached pandas-1.1.5-cp38-cp38-manylinux1_x86_64.whl (9.3 MB)
  Using cached pytz-2020.4-py2.py3-none-any.whl (509 kB)


In [46]:
import json
import os

import pandas as pd
import numpy as np

In [3]:
SLUGA_NARODA_PARRY = "СН"
OPZG_PARTY = "ОПЗЖ"
ES_PARTY = "ЕС"
GOLOS_PARTY = "Голос"
BATKIVSHCHINA_PARTY = "Батьківщина"
ZA_MAYBUTNE_PARTY = "За Майбутне"
DOVIRA_PARTY = "Довіра"
POZAFRACTION_PARTY = "Позафракційні"

На базе каталога из п.2 соберем
* всех депутатов (и их последний актуальный статус)
* все возможные варианты (не)голосования

К сожалению, ФИО депутата не является уникальным ключем -- в Раде есть полные тезки,а именно Ковальов О.І. (СН) и Ковальов О.І. (Позафракційний и немного позже -- Довіра)

Нас будет интересовать крайняя партийная принадлежность, то, в какой партии/объединении депутат находится на сейчас

In [31]:
all_names = []
all_options = []

all_deputy_profile = []
all_options_profile = []


all_deputy_profile.append(
    {
        "name": "Ковальов О.І.",
        "party": SLUGA_NARODA_PARRY
    }
)
all_deputy_profile.append(
    {
        "name": "Ковальов О.І.",
        "party": DOVIRA_PARTY
    }
)
all_names.append("Ковальов О.І.")

for path in sorted(list([int(x) for x in os.listdir("./deputy_vote_result")]), reverse=True):
    file = open(f"./deputy_vote_result/{path}/result.json", "r")
    text = file.read()

    json_info = json.loads(text)

    for vote in json_info:
        dep_profile = {
            "name": vote['name'],
            "party": vote['party']
        }

        if vote['name'] not in all_names:
            all_names.append(vote['name'])
            all_deputy_profile.append(dep_profile)

        if vote['result'] not in all_options:
            all_options.append(vote['result'])

            all_options_profile.append(
                {
                    "result": vote['result'],
                }
            )

    file.close()


deputy_df = pd.DataFrame(all_deputy_profile)
deputy_df.to_csv('1_deputy_profile.csv', index_label='id')

decision_df = pd.DataFrame(all_options_profile)
decision_df.to_csv('1_decision_profile.csv', index_label='id')

На основе полученных данных, соберем матрицу голосований "все за всё" (не забывая про Ковальова)

In [32]:
all_result = []

deputy_dict = dict(zip(deputy_df['name'], deputy_df.index))
decision_dict = dict(zip(decision_df['result'], decision_df.index))

for path in os.listdir("./deputy_vote_result"):
    file = open(f"./deputy_vote_result/{path}/result.json", "r")
    text = file.read()
    json_info = json.loads(text)

    current_vote_dict = dict()
    current_vote_dict['law_id'] = int(path)

    for person in json_info:
        if person['name'] != "Ковальов О.І.":
            current_vote_dict[deputy_dict[person['name']]] = decision_dict[person['result']]
        else:
            if person['party'] == SLUGA_NARODA_PARRY:
                current_vote_dict[0] = decision_dict[person['result']]
            else:
                current_vote_dict[1] = decision_dict[person['result']]

    all_result.append(current_vote_dict)
    file.close()


deputy_vote_df = pd.DataFrame(all_result)
deputy_vote_df = deputy_vote_df.set_index('law_id')
deputy_vote_df.to_csv('2_deputy_vote_result.csv', index_label="law_id")

Заметим, что выбор депутата "гендерно окрашен", исправим это в данных + заполним некоторой меткой отсутствие данных в системе о голосовании конкретного депутата (умер, сдал мандат и.т.д.)

In [33]:
decision_df

,result
0,Відсутній
1,За
2,Утримався
3,Утрималась
4,Відсутня
5,Не голосувала
6,Не голосував
7,Проти


In [34]:
for column in deputy_vote_df.columns:
    deputy_vote_df[column] = deputy_vote_df[column].fillna(-1)
    deputy_vote_df[column].replace([2], 3, inplace=True)
    deputy_vote_df[column].replace([0], 4, inplace=True)
    deputy_vote_df[column].replace([6], 5, inplace=True)
    
deputy_vote_df = deputy_vote_df.astype(int)

In [35]:
deputy_vote_df

,2,3,4,423,5,6,7,8,9,10,...,416,58,324,384,433,434,435,436,437,438
law_id,,,,,,,,,,,,,,,,,,,,,
490,1,1,1,1,4,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4367,3,3,3,1,4,5,5,5,3,3,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6084,1,1,1,1,4,1,1,1,1,1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4988,4,7,7,4,4,5,3,5,3,7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2884,1,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,1,4,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3675,3,1,3,3,4,5,3,4,3,3,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1895,1,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [36]:
deputy_vote_df.to_csv('3_deputy_vote_result_filter.csv', index_label="law_id")

Удалим всех депутатов, увы, сошедших с дистанции раньше положенного, транспонируем матрицу и построим матрицу близости (proximity matrix)

In [37]:
deputy_vote_df = deputy_vote_df.T
deputy_vote_df = deputy_vote_df.loc[deputy_vote_df.min(axis=1) >= 0]
deputy_vote_df

law_id,490,4367,6084,4988,2884,4244,3027,1942,720,8683,...,4204,3865,2881,5178,5390,1697,3675,1895,2332,1560
2,1,3,1,4,1,3,1,1,7,1,...,3,3,1,1,4,1,3,1,1,1
3,1,3,1,7,1,7,1,1,5,4,...,7,7,1,1,1,4,1,1,3,1
4,1,3,1,7,1,3,1,1,3,1,...,3,3,1,1,1,1,3,1,4,1
5,4,4,4,4,1,4,1,1,5,4,...,5,7,1,1,1,1,4,1,3,1
6,1,5,1,5,1,7,1,1,4,1,...,3,3,1,1,1,1,5,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,3,3,1,3,5,3,1,1,3,3,...,1,3,1,1,1,1,3,1,1,1
404,5,5,1,4,3,5,3,1,5,3,...,1,5,1,4,1,1,5,5,1,1
418,5,4,4,4,4,4,3,5,5,4,...,4,4,4,4,4,5,4,3,5,4
419,5,1,1,4,5,5,3,3,5,4,...,1,4,5,3,3,5,4,5,5,3


In [41]:
deputy_vote_df.to_csv('4_deputy_vote_result_T.csv', index_label="dep_id")

Введем две разные функции для построения матрицы близости -- мягкую -- (которая считает сходством только одинаковое действие -- За/Против/Воздержался/Не голосовал) и более строгую, которая помимо прочего считает за сходство еще и одновременное отсутствие депутата 

In [48]:
deputy_vote_df_1 = deputy_vote_df.replace([0], -1)
deputy_vote_df_2 = deputy_vote_df.replace([0], -2)

def get_strict_proximity(i, j):
    return np.sum(deputy_vote_df.iloc[i] == deputy_vote_df.iloc[j])


def get_low_proximity(i, j):
    if i == j:
        return np.sum(deputy_vote_df.iloc[i] == deputy_vote_df.iloc[j])
    else:
        return np.sum(deputy_vote_df_1.iloc[i] == deputy_vote_df_2.iloc[j])

In [53]:
# long chunk -- calculate the results in around a minute

rows = deputy_vote_df.shape[0]

proximity_matrix = np.zeros((rows, rows))
low_proximity_matrix = np.zeros((rows, rows))

for i in range(rows):
    print(f"{i}/{rows}")
    for j in range(rows):
        proximity_matrix[i, j] = get_strict_proximity(i, j)
        low_proximity_matrix[i, j] = get_low_proximity(i, j)


proximity_df = pd.DataFrame(proximity_matrix)
proximity_df.columns = deputy_vote_df.index
proximity_df['dep_id'] = deputy_vote_df.index
proximity_df = proximity_df.set_index('dep_id')


low_proximity_df = pd.DataFrame(low_proximity_matrix)
low_proximity_df.columns = deputy_vote_df.index
low_proximity_df['dep_id'] = deputy_vote_df.index
low_proximity_df = low_proximity_df.set_index('dep_id')

proximity_df.to_csv('5_deputy_vote_strict_proximity_matrix.csv', index_label='dep_id')
low_proximity_df.to_csv('5_deputy_vote_low_proximity_matrix.csv', index_label='dep_id')

0/404
1/404
2/404
3/404
4/404
5/404
6/404
7/404
8/404
9/404
10/404
11/404
12/404
13/404
14/404
15/404
16/404
17/404
18/404
19/404
20/404
21/404
22/404
23/404
24/404
25/404
26/404
27/404
28/404
29/404
30/404
31/404
32/404
33/404
34/404
35/404
36/404
37/404
38/404
39/404
40/404
41/404
42/404
43/404
44/404
45/404
46/404
47/404
48/404
49/404
50/404
51/404
52/404
53/404
54/404
55/404
56/404
57/404
58/404
59/404
60/404
61/404
62/404
63/404
64/404
65/404
66/404
67/404
68/404
69/404
70/404
71/404
72/404
73/404
74/404
75/404
76/404
77/404
78/404
79/404
80/404
81/404
82/404
83/404
84/404
85/404
86/404
87/404
88/404
89/404
90/404
91/404
92/404
93/404
94/404
95/404
96/404
97/404
98/404
99/404
100/404
101/404
102/404
103/404
104/404
105/404
106/404
107/404
108/404
109/404
110/404
111/404
112/404
113/404
114/404
115/404
116/404
117/404
118/404
119/404
120/404
121/404
122/404
123/404
124/404
125/404
126/404
127/404
128/404
129/404
130/404
131/404
132/404
133/404
134/404
135/404
136/404
137/404
138/40

In [54]:
proximity_df

,2,3,4,5,6,7,8,9,10,11,...,400,402,414,415,417,403,404,418,419,421
dep_id,,,,,,,,,,,,,,,,,,,,,
2,5828.0,2948.0,3842.0,2847.0,3462.0,3708.0,3240.0,3658.0,3903.0,3501.0,...,1795.0,2254.0,3625.0,1738.0,3761.0,2352.0,1704.0,1097.0,1015.0,2119.0
3,2948.0,5828.0,3454.0,3012.0,3089.0,3238.0,3064.0,3174.0,3513.0,3111.0,...,1828.0,1901.0,3223.0,1496.0,3346.0,2005.0,1503.0,1045.0,1020.0,1688.0
4,3842.0,3454.0,5828.0,3072.0,3943.0,4127.0,3622.0,4277.0,4548.0,3936.0,...,2103.0,2507.0,4077.0,1810.0,4396.0,2711.0,1493.0,498.0,676.0,1890.0
5,2847.0,3012.0,3072.0,5828.0,3158.0,3092.0,3100.0,2861.0,3143.0,2936.0,...,1914.0,1947.0,3209.0,1607.0,2913.0,1929.0,1878.0,1884.0,1578.0,1637.0
6,3462.0,3089.0,3943.0,3158.0,5828.0,3785.0,3387.0,3680.0,3904.0,3531.0,...,2132.0,2273.0,4000.0,1646.0,3692.0,2379.0,1613.0,912.0,1007.0,1629.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,2352.0,2005.0,2711.0,1929.0,2379.0,2628.0,2255.0,2760.0,2788.0,2339.0,...,2670.0,3338.0,2651.0,2383.0,2729.0,5828.0,2629.0,703.0,1236.0,2273.0
404,1704.0,1503.0,1493.0,1878.0,1613.0,1673.0,1697.0,1498.0,1510.0,1439.0,...,2612.0,2290.0,1685.0,1630.0,1250.0,2629.0,5828.0,1474.0,2235.0,1779.0
418,1097.0,1045.0,498.0,1884.0,912.0,521.0,862.0,621.0,289.0,875.0,...,1136.0,818.0,446.0,1044.0,626.0,703.0,1474.0,5828.0,3035.0,1380.0
